# **Text Clustering**

## **Imports** 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import umap.umap_ as umap
import math
import os
import warnings

import tensorflow as tf
import keras.backend as K

import nltk
nltk.download('punkt')

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras.utils import plot_model
from keras.layers import Dense, Input
from keras.models import Model
from keras.initializers import VarianceScaling, glorot_uniform

# random_state: mutual_info_classif, KMeans, MiniBatchKMeans, SpectralClustering, SpectralBiclustering, SpectralCoclustering, NMF, silhouette_score, VarianceScaling, glorot_uniform
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, mutual_info_classif 
from sklearn.cluster import KMeans, MiniBatchKMeans, SpectralClustering, SpectralBiclustering, SpectralCoclustering, AgglomerativeClustering, Birch
from sklearn.decomposition import NMF
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, homogeneity_score, adjusted_mutual_info_score, silhouette_score, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

from scipy import sparse
from scipy.optimize import linear_sum_assignment as linear_assignment

from IPython.display import Image

from tqdm import tqdm

tf.random.set_seed(10)
np.random.seed(10)
random_state = np.random.RandomState(seed=10)

warnings.filterwarnings("ignore")

## **Metrics** 

In [ ]:
nmi_fun = normalized_mutual_info_score
ari_fun = adjusted_rand_score
ami_fun = adjusted_mutual_info_score
silhouette_score_fun = silhouette_score

# acc_fun = homogeneity_score

def acc_fun (y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed

    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = list(linear_assignment(w.max() - w))
    acc_sum = 0
    for k in range(len(w)):
       acc_sum =  w[ind[0][k],ind[1][k]] + acc_sum
    return acc_sum * 1.0 / y_pred.size

## Dataset Import

### WEBKB

In [ ]:
def fetching_data_set():
    data = []
    f = open("../data/webkb/webkb-all-stemmed.txt", "r")
    for x in f:
        data.append(x)
    f.close()
    print('Number of documents: ', len(data))
    return data

data = fetching_data_set()
raw_data = []
target = []
for i in data:
  ff = i.split("\t")
  target.append(ff[0])
  raw_data.append(ff[1])

max_features = 2000

# x = CountVectorizer(dtype=np.float64, max_features=max_features).fit_transform(raw_data)
x = CountVectorizer(dtype=np.float64).fit_transform(raw_data)
x = TfidfTransformer(norm='l2', sublinear_tf=True).fit_transform(x)
x = np.asarray(x.todense()) * np.sqrt(x.shape[1])

print('todense succeed')

y = []
for i in target:
  if i == "student":
    y.append(0)
  elif i == "faculty":
    y.append(1)
  elif i == "project":
    y.append(2)
  elif i == "course":
    y.append(3)

y = np.asarray(y)

# p = random_state.permutation(x.shape[0])
# x, y = x[p], y[p]
# print('permutation finished')
n_clusters = 4

# for i in range(x.shape[0]):
#   x[i] = (x[i]-np.min(x[i]))/(np.max(x[i])-np.min(x[i]))
main_inputs = x
x.shape

### Reutersidf10k

In [ ]:
# def load_reuters(max_features, data_path='../data/reuters'):
#     print('making reuters idf features')
#     make_reuters_data(data_path, max_features)
#     print('reutersidf saved to ' + data_path)
#     data = np.load(os.path.join(data_path, 'reutersidf10k.npy'), allow_pickle=True).item()
#     # has been shuffled
#     x = data['data']
#     y = data['label']
#     x = x.reshape((x.shape[0], -1)).astype('float32')
#     y = y.reshape((y.size,))
#     #print('REUTERSIDF10K samples', x.shape)
#     return x, y


# def make_reuters_data(data_dir, max_features):
#     did_to_cat = {}
#     cat_list = ['CCAT', 'GCAT', 'MCAT', 'ECAT']
#     with open(os.path.join(data_dir, 'rcv1-v2.topics.qrels')) as fin:
#         for line in fin.readlines():
#             line = line.strip().split(' ')
#             cat = line[0]
#             did = int(line[1])
#             if cat in cat_list:
#                 did_to_cat[did] = did_to_cat.get(did, []) + [cat]
#         for did in list(did_to_cat.keys()):
#             if len(did_to_cat[did]) > 1:
#                 del did_to_cat[did]

#     dat_list = ['lyrl2004_tokens_test_pt0.dat',
#                 'lyrl2004_tokens_test_pt1.dat',
#                 'lyrl2004_tokens_test_pt2.dat',
#                 'lyrl2004_tokens_test_pt3.dat',
#                 'lyrl2004_tokens_train.dat']
#     data = []
#     target = []
#     cat_to_cid = {'CCAT': 0, 'GCAT': 1, 'MCAT': 2, 'ECAT': 3}
#     del did
#     for dat in dat_list:
#         with open(os.path.join(data_dir, dat)) as fin:
#             for line in fin.readlines():
#                 if line.startswith('.I'):
#                     if 'did' in locals():
#                         assert doc != ''
#                         if did in did_to_cat:
#                             data.append(doc)
#                             target.append(cat_to_cid[did_to_cat[did][0]])
#                     did = int(line.strip().split(' ')[1])
#                     doc = ''
#                 elif line.startswith('.W'):
#                     assert doc == ''
#                 else:
#                     doc += line

#     assert len(data) == len(did_to_cat)

#     x = CountVectorizer(dtype=np.float64, max_features=max_features).fit_transform(data)
#     # x = CountVectorizer(dtype=np.float64).fit_transform(data)
#     print(x.shape)
#     y = np.asarray(target)

#     x = TfidfTransformer(norm='l2', sublinear_tf=True).fit_transform(x)
#     x, y = x[:10000], y[:10000]
#     x = np.asarray(x.todense()) * np.sqrt(x.shape[1])
#     print('todense succeed')

#     p = random_state.permutation(x.shape[0])
#     x, y = x[p], y[p]
#     print('permutation finished')

#     assert x.shape[0] == y.shape[0]
#     x = x.reshape((x.shape[0], -1))
#     print(x.shape)
#     np.save(os.path.join(data_dir, 'reutersidf10k.npy'), {'data': x, 'label': y})

# max_features = 500
# x,y = load_reuters(max_features=max_features)
# n_clusters = 2

# for i in range(x.shape[0]):
#     x[i] = (x[i]-np.min(x[i]))/(np.max(x[i])-np.min(x[i]))
# main_inputs = x
# x.shape

### 20newsgroups

In [ ]:
# newsgroups = fetch_20newsgroups(subset='all')
# y = np.asarray(newsgroups.target)

# max_features = 2000

# x = CountVectorizer(dtype=np.float64, max_features=max_features).fit_transform(newsgroups.data)
# # x = CountVectorizer(dtype=np.float64).fit_transform(newsgroups.data)
# x = TfidfTransformer(norm='l2', sublinear_tf=True).fit_transform(x)
# x = np.asarray(x.todense()) * np.sqrt(x.shape[1])
# print('todense succeed')

# p = random_state.permutation(x.shape[0])
# x, y = x[p], y[p]
# print('permutation finished')
# n_clusters = 2
# main_inputs = x
# x.shape

## Laplacian Matrix

In [ ]:
#measure distance between vectors v and w
def calculate_hamming_distance(x):
    hamming_distance_matrix = np.zeros((len(x),len(x)))
    for i in tqdm(range(0, len(x))):
        for j in range(0, len(x)):
            if(hamming_distance_matrix[i][j]!=0 or hamming_distance_matrix[j][i]!=0):
                continue
            v = x[i]
            w = x[j]

            v_sum = np.sum(v)
            w_sum = np.sum(w)

            temp = 1-np.sign(v*w)
            v_numerator = np.sum(v*temp)
            w_numerator = np.sum(w*temp)

            hamming_distance_matrix[i][j] = (v_numerator/v_sum) + (w_numerator/w_sum)
            hamming_distance_matrix[j][i] = (v_numerator/v_sum) + (w_numerator/w_sum)
    
    return hamming_distance_matrix

w = cosine_similarity(x)
print(w,"\n-----",w.shape)

hamming_distance = calculate_hamming_distance(x)
print(hamming_distance,"\n-----",hamming_distance.shape)

w = w / (np.square(hamming_distance) + w)
print(w,"\n-----",w.shape)

for i in range(len(w)):
    w[i,i] = 0

for i in range(len(w)):
    s_w = sorted(w[i])[-10:]
    for j in range(len(w[i])):
        if w[i,j] not in s_w:
            w[i,j] = 0

D = np.zeros(w.shape)
for i in range(len(w)):
    D[i, i] = np.count_nonzero(w[i])

laplacian_matrix = np.zeros(w.shape)

for i in range(len(laplacian_matrix)):
    for j in range(len(laplacian_matrix)):
        if i == j and D[i, j] != 0:
            laplacian_matrix[i, j] = 1
        elif i != j and w[i, j] != 0:
            laplacian_matrix[i, j] = (-1)/np.sqrt(D[i, i] * D[j, j])

## Other Methods

In [ ]:
def pre_processing(data):
    porter = PorterStemmer()
    vectorizer = TfidfVectorizer(stop_words='english')

    # Aggregating and vectorizing all texts
    all_text = [" ".join(data)]
    X = vectorizer.fit_transform(all_text)
    print('Dimension of vectorizing aggregated texts without stemming: ', X.shape)

    # # Opening dictionary
    # f = open('words.txt', 'r')
    # dictionary = f.read().split()
    # f.close()

    # # Filtering terms via dictionary and stemming
    # token_words = terms
    # print('All terms: ', len(token_words))
    # # stem_sentence = [porter.stem(word) for word in token_words if word in dictionary]
    # stem_sentence = [porter.stem(word) for word in token_words]
    #
    # all_text_stem = [" ".join(stem_sentence)]
    # print('Number of terms after filtering dictionary: ', len(stem_sentence))
    #
    # # Vectorizing aggregated texts after filtering and stemming
    # X = vectorizer.fit_transform(all_text_stem)
    # print('Dimension of vectorizing after filtering and stemming: ', X.shape)

    # Stemming texts separately and vectorizing
    texts_stem = []
    target = []
    for i in range(len(data)):
        token_words = word_tokenize(data[i])
        stem_sentence = []
        target.append("".join(token_words[0]))
        for word in token_words[1:]:
            stem_sentence.append(porter.stem(word.lower()))
        texts_stem.append(" ".join(stem_sentence))

    X = vectorizer.fit_transform(texts_stem)
    print('Dimension of tf-idf matrix ', X.shape)

    return X, target


def feature_selection(X, features_number):
    # Feature selection using mutual information
    X_new = SelectKBest(mutual_info_classif, k=features_number, random_state=10).fit_transform(X, target)
    print('Dimension of tf-idf matrix after mutual information: ', X_new.shape)
    return X_new


def removing_zero_rows(X_new, target):
    # Removing zero rows
    target_new = target
    X_new2 = X_new.todense()
    l = np.where(X_new2.sum(axis=1) != 0)[0]
    X_new = X_new2[l, :]
    index = []
    c = 0
    j = 0
    for i in range(X_new2.shape[0]):
        if X_new2[i].sum() == 0:
            target_new = np.delete(target_new, j)
            index.append(j)
            j = j - 1
            c = c + 1
        j = j + 1
    X_new = sparse.csr_matrix(X_new)
    print('The number of removed documents: ', c, X_new.shape)
    return X_new, target_new


def compared_methods(clusters, X_new, target_new):
    # Clustering
    kmeans = KMeans(n_clusters=clusters, max_iter=200, random_state=10).fit(X_new)
    print('KMeans NMI: ', normalized_mutual_info_score(kmeans.labels_, target_new))
    print('KMeans AMI: ', adjusted_mutual_info_score(kmeans.labels_, target_new))
    print('KMeans Silhouette: ', silhouette_score(X_new, kmeans.labels_, random_state=10))

    model = NMF(n_components=clusters, init='random', random_state=10)
    D = model.fit_transform(X_new)
    W = model.components_
    NMF_labels = np.argmax(D, axis=1)
    print('NMF NMI: ', normalized_mutual_info_score(NMF_labels, target_new))
    print('NMF AMI: ', adjusted_mutual_info_score(NMF_labels, target_new))
    print('NMF Silhouette: ', silhouette_score(X_new, NMF_labels, random_state=10))

    model = NMF(n_components=clusters, init='random', solver='mu', beta_loss='kullback-leibler', random_state=10)
    D = model.fit_transform(X_new)
    W = model.components_
    NMF_KL_labels = np.argmax(D, axis=1)
    print('NMF_KL NMI: ', normalized_mutual_info_score(NMF_KL_labels, target_new))
    print('NMF_KL AMI: ', adjusted_mutual_info_score(NMF_KL_labels, target_new))
    print('NMF_KL Silhouette: ', silhouette_score(X_new, NMF_KL_labels, random_state=10))

    # Spectral = SpectralClustering(n_clusters=clusters, random_state=10).fit(X_new)
    # print('Spectral NMI: ', normalized_mutual_info_score(Spectral.labels_, target_new))
    # print('Spectral AMI: ', adjusted_mutual_info_score(Spectral.labels_, target_new))
    # print('Spectral Silhouette: ', silhouette_score(X_new, Spectral.labels_, random_state=10))

    # SpectralBi = SpectralBiclustering(n_clusters=clusters, random_state=10).fit(X_new)
    # print('SpectralBi NMI: ', normalized_mutual_info_score(SpectralBi.row_labels_, target_new))
    # print('SpectralBi AMI: ', adjusted_mutual_info_score(SpectralBi.row_labels_, target_new))
    # print('SpectralBi Silhouette: ', silhouette_score(X_new, SpectralBi.row_labels_, random_state=10))

    # SpectralCo = SpectralCoclustering(n_clusters=clusters, random_state=10).fit(X_new)
    # print('SpectralCo NMI: ', normalized_mutual_info_score(SpectralCo.row_labels_, target_new))
    # print('SpectralCo AMI: ', adjusted_mutual_info_score(SpectralCo.row_labels_, target_new))
    # print('SpectralCo Silhouette: ', silhouette_score(X_new, SpectralCo.row_labels_, random_state=10))

    Agglomerative = AgglomerativeClustering(n_clusters=clusters).fit(X_new)
    print('Agglomerative NMI: ', normalized_mutual_info_score(Agglomerative.labels_, target_new))
    print('Agglomerative AMI: ', adjusted_mutual_info_score(Agglomerative.labels_, target_new))
    print('Agglomerative Silhouette: ', silhouette_score(X_new, Agglomerative.labels_, random_state=10))

    brc = Birch(n_clusters=clusters).fit(X_new)
    labels = brc.predict(X_new)
    print('Birch NMI: ', normalized_mutual_info_score(labels, target_new))
    print('Birch AMI: ', adjusted_mutual_info_score(labels, target_new))
    print('Birch Silhouette: ', silhouette_score(X_new, labels, random_state=10))

    kmeansp = MiniBatchKMeans(n_clusters=clusters, random_state=10).fit(X_new)
    print('MiniBatchKMeans NMI: ', normalized_mutual_info_score(kmeansp.labels_, target_new))
    print('MiniBatchKMeans AMI: ', adjusted_mutual_info_score(kmeansp.labels_, target_new))
    print('MiniBatchKMeans Silhouette: ', silhouette_score(X_new, kmeansp.labels_, random_state=10))


def similarity(X_new):
    # Computing similarity
    S = sparse.csr_matrix(cosine_similarity(X_new))
    return S


def RANMF(clusters, X_new, target_new, S):
    #### The Proposed RANMF
    Lambeda = 0.05
    sumS = np.squeeze(np.array(S.sum(axis=1)))
    D = random_state.rand(X_new.shape[0], clusters)
    W = random_state.rand(clusters, X_new.shape[1])
    for ite in range(200):
        # print('Iteration: ', ite)
        DW = D @ W
        SD = S @ D

        DW[DW <= .000001] = .000001
        XDW = X_new / DW
        XDWW = XDW @ W.transpose()
        sumW = W.sum(axis=1)
        M = []
        for f in range(clusters):
            M.append(sumW[f] + (2 * Lambeda * D[:, f] * sumS))
        M = np.squeeze(M).transpose()
        M[M == 0] = .000001
        D = D * np.asarray(XDWW + (2 * Lambeda * SD)) / M
        ####################
        DW = D @ W
        DW[DW <= .000001] = .000001
        XDW = X_new / DW
        XDWD = D.transpose() @ XDW
        sumD = D.sum(axis=0)
        W = W * np.asarray(XDWD)
        T = []
        for f in range(clusters):
            if sumD[f] != 0:
                T.append(W[f, :] / sumD[f])
        T = np.squeeze(T)
        W = T

    RANMF_labels = np.argmax(D, axis=1)
    print('RANMF NMI: ', normalized_mutual_info_score(RANMF_labels, target_new))
    print('RANMF AMI: ', adjusted_mutual_info_score(RANMF_labels, target_new))
    print('RANMF Silhouette: ', silhouette_score(X_new, RANMF_labels, random_state=10))


def RANMF_converge(clusters, X_new, target_new, S):
    #### The Proposed RANMF
    Lambeda = 0.05
    converge = []
    before = 0
    X_new2 = X_new
    X_new2[X_new2 == 0] = .000001
    X_new2 = np.array(X_new2)
    sumS = np.squeeze(np.array(S.sum(axis=1)))
    D = random_state.rand(X_new.shape[0], clusters)
    W = random_state.rand(clusters, X_new.shape[1])
    for ite in range(200):
        # print('Iteration: ', ite)
        DW = D @ W
        SD = S @ D

        DW[DW <= .000001] = .000001
        XDW = X_new / DW
        XDWW = XDW @ W.transpose()
        sumW = W.sum(axis=1)
        M = []
        for f in range(clusters):
            M.append(sumW[f] + (2 * Lambeda * D[:, f] * sumS))
        M = np.squeeze(M).transpose()
        M[M == 0] = .000001
        D = D * np.asarray(XDWW + (2 * Lambeda * SD)) / M
        ####################
        DW = D @ W
        DW[DW <= .000001] = .000001
        XDW = X_new / DW
        XDWD = D.transpose() @ XDW
        sumD = D.sum(axis=0)
        W = W * np.asarray(XDWD)
        T = []
        for f in range(clusters):
            if sumD[f] != 0:
                T.append(W[f, :] / sumD[f])
        T = np.squeeze(T)
        W = T

        # # Convergence
        DW[DW < .000001] = .000001
        cost = (X_new2 * np.array(np.log(X_new2 / DW)) - X_new2 + DW).sum()
        # print('Iteration: ', ite)
        # cost = cost + Lambeda * ((euclidean_distances(D, D) ** 2) * np.array(S.todense())).sum()

        # if (before - cost) < 0:
        # print(' Not converge: ', ite)
        before = cost
        print(cost)
        converge.append(cost)


clusters = n_clusters


# # With feature selection
# X_new = feature_selection(X, features_number)

# # Without feature selection
# X_new = X

# X_new, target_new = removing_zero_rows(X_new, target)
X_new = x
target_new = y
compared_methods(clusters, X_new, target_new)

S = similarity(X_new)

RANMF(clusters, X_new, target_new, S)

# RANMF_converge(clusters, X_new, target_new, S)

## **Autoencoder and Encoder**

In [ ]:
def autoencoder(dims, act='relu', init='glorot_uniform'):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
            The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden and Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1
    # input
    input_img = Input(shape=(dims[0],), name='input')
    x = input_img
    # internal layers in encoder
    for i in range(n_stacks-1):
        x = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(x)

    # hidden layer
    encoded = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)  # hidden layer, features are extracted from here

    x = encoded
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        x = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)

    # output
    x = Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)
    decoded = x
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=encoded, name='encoder')


## Hyper-params

In [ ]:
dims = [x.shape[-1], 500, 500, 2000, 10]
init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform', seed=10)
pretrain_optimizer = SGD(learning_rate=1, momentum=0.9)
pretrain_epochs = 500
batch_size = 64
save_dir = 'clustering'

In [ ]:
autoencoder, encoder = autoencoder(dims, init=init)

In [ ]:
# plot_model(autoencoder, to_file='autoencoder.png', show_shapes=True)
# Image(filename='autoencoder.png')

In [ ]:
# plot_model(encoder, to_file='encoder.png', show_shapes=True)
# Image(filename='encoder.png') 

## Pretrain auto-encoder

In [ ]:
autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')
autoencoder.fit(x, x, batch_size=256, epochs=pretrain_epochs - 300) #, callbacks=cb)
# # autoencoder.save_weights('/content/clustering/MyDrive/clustering/ae_augmentation_weights_20NEWSGROUPS.h5')
# # autoencoder.save_weights('/content/clustering/MyDrive/clustering/ae_augmentation_weights_20NEWSGROUPS.h5')
autoencoder.save_weights('clustering/ae_augmentation_weights_WEBKB.h5')
#autoencoder.save_weights('clustering/ae_augmentation_weights_WEBKB_500.h5')

### Load the pre-trained auto encoder weights

In [ ]:
# autoencoder.load_weights('/content/clustering/MyDrive/clustering/ae_augmentation_weights_20NEWSGROUPS.h5')
# autoencoder.load_weights('/content/clustering/MyDrive/clustering/ae_augmentation_weights_REUTERSIDF10K.h5')
autoencoder.load_weights('clustering/ae_augmentation_weights_WEBKB.h5')
#autoencoder.load_weights('clustering/ae_augmentation_weights_WEBKB_500.h5')

## Build clustering model


### Clusteringand embedding layers

In [ ]:
class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer=glorot_uniform(seed=10), name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
         Measure the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """

        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [ ]:
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
Clustering_Loss_layer = tf.keras.layers.Concatenate(axis=1, name='Clustering_Loss')([clustering_layer, encoder.output])
Recunstruction_Loss_layer  = tf.keras.layers.Concatenate(axis=1, name='Recunstruction_Loss')([autoencoder.output, encoder.output])

Clustering_Loss_layer.trainable=False
Recunstruction_Loss_layer.trainable=False

model = Model(inputs=encoder.input, outputs=[Clustering_Loss_layer, Recunstruction_Loss_layer])

In [ ]:
layer_names = [layer.name for layer in model.layers]

In [ ]:
print(layer_names)

In [ ]:
# plot_model(model, to_file='model.png', show_shapes=True)
# Image(filename='model.png') 

### Step 1: initialize cluster centers using k-means

In [ ]:
kmeans = KMeans(n_clusters=n_clusters, n_init=250, random_state=10)
y_pred = kmeans.fit_predict(encoder.predict(x))

In [ ]:
y_pred_last = np.copy(y_pred)

In [ ]:
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

### Step 2: deep clustering
Compute p_i by first raising q_i to the second power and then normalizing by frequency per cluster:

In [ ]:
# computing an auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [ ]:
loss = 0
index = 0
#maxiter = 8000
maxiter = 1000
update_interval = 3
index_array = np.arange(x.shape[0])
tol = 0.001 # tolerance threshold to stop training

### Start training

In [ ]:
def Clustering_Loss(y_true, y_pred):

    kld_Loss = tf.keras.losses.kld(y_true[:,:n_clusters], y_pred[:,:n_clusters])

    # calculate pairwise distance matrix
    # pairwise_diff = K.expand_dims(y_pred[:,n_clusters:], 0) - K.expand_dims(y_pred[:,n_clusters:], 1)
    # pairwise_squared_distance = K.sum(K.square(pairwise_diff), axis=-1)
    # pairwise_distance = K.sqrt(pairwise_squared_distance + K.epsilon())

    # calculate Cons distance matrix
    # Cons_Simi = similarityMatrix.Calculate_Similarity_Matrix(4,x[idx])

    return (1/len(idx) * K.sum(kld_Loss))
    #return (1/len(idx) * K.sum(kld_Loss)) + 0.0001 * 1/len(idx) * K.sum(pairwise_distance * Cons_Simi)

In [ ]:
Z = encoder.output
Z

In [ ]:
def Manifold_Loss():
  this_batch_size = len(idx)
  
  this_batch_laplacian_matrix = np.zeros((this_batch_size,this_batch_size))
  
  for i in range(0,len(idx)):
    for j in range(0,len(idx)):
      this_batch_laplacian_matrix[i][j] = laplacian_matrix[idx[i]][idx[j]]

  this_batch_inputs = np.asarray([main_inputs[i] for i in idx])
  #print("this_batch_inputs: ", this_batch_inputs,"\n shape: ", this_batch_inputs.shape, "\n type: ", type(this_batch_inputs))


  this_batch_laplacian_matrix = tf.cast(this_batch_laplacian_matrix, tf.float32)

  lambda_hyperparameter = 100 # 100

  #extract Z(latent) to compute Manifold_Loss
  #Z = encoder.output

  #convert keras tensor to numpy array
  inp = encoder.input
  output = encoder.output
  functor = K.function([inp], [output])
  Z = functor([this_batch_inputs])[0]

  #print("type(Z):", type(Z))
  #print("-----------")
  #print(Z)

  #Z_Transpose
  Z_Transpose = np.transpose(Z)
  
  #print for debugging purposes
  #print("type(Z_Transpose): ", type(Z_Transpose), " Shape: ", Z_Transpose.shape)
  #print("type(Z): ",type(Z), " Shape: ", Z.shape)
  #print("type(this_batch_laplacian_matrix): ", type(this_batch_laplacian_matrix), " Shape: ", this_batch_laplacian_matrix.shape)
  

  temp = tf.tensordot(Z_Transpose, this_batch_laplacian_matrix, axes=1)
  #temp = K.dot(Z_Transpose, this_batch_laplacian_matrix)
  temp = tf.tensordot(temp, Z, axes=1)
  #temp = K.dot(temp, Z)

  #print("final_matrix: ", temp, "\n shape: ", temp.shape, "\n type: ", type(temp))
  
  #computing trace
  #main_diagonal = [temp[i][i] for i in range(0,temp.shape[0])]
  #print("main_diagonal: ", main_diagonal)
  #temp_trace = K.sum(main_diagonal)

  temp_trace = np.trace(temp)
  
  #print("trace: ", temp_trace)

  manifold_loss = lambda_hyperparameter * temp_trace
  #print("manifold_loss: ", manifold_loss)

  return manifold_loss

In [ ]:
def elastic_loss(batch_indexes, delta=0): # 0.001
  
  batch = x[batch_indexes]
  #convert keras tensor to numpy array
  inp = autoencoder.input
  output = autoencoder.output
  functor = K.function([inp], [output])
  batch_output = functor([batch])[0]

  l1_norm = np.sum(np.abs(batch - batch_output), axis=1)

  l2_pseudo_loss = np.sum((delta * l1_norm ** 2)/(delta + l1_norm))
  l1_pseudo_loss = np.sum((l1_norm ** 2)/(delta + l1_norm))

  e_loss = l2_pseudo_loss + l1_pseudo_loss  
    
  # print(f"elastic loss: {e_loss}")

  return e_loss

In [ ]:
def sparsity_loss(gamma=0.005): # 0.005
  this_batch_size = len(idx)
  
  this_batch_laplacian_matrix = np.zeros((this_batch_size,this_batch_size))
  
  for i in range(0,len(idx)):
    for j in range(0,len(idx)):
      this_batch_laplacian_matrix[i][j] = laplacian_matrix[idx[i]][idx[j]]

  this_batch_inputs = np.asarray([main_inputs[i] for i in idx])

  this_batch_laplacian_matrix = tf.cast(this_batch_laplacian_matrix, tf.float32)

  #convert keras tensor to numpy array
  inp = encoder.input
  output = encoder.output
  functor = K.function([inp], [output])
  Z = functor([this_batch_inputs])[0]

  Z_transpose_norm = np.sum(np.square(np.sum(np.abs(Z), axis=0)))
  
  s_loss = gamma * Z_transpose_norm
  # print("sparsity_loss: ", s_loss)

  return s_loss

In [ ]:
def total_loss(y_true, y_pred):
    t_loss = elastic_loss(idx) + Manifold_Loss() + sparsity_loss()
    print(f"total loss: {t_loss}")
    return t_loss

In [ ]:
model.compile(loss=[Clustering_Loss, total_loss], loss_weights=[0.1, 1], optimizer=pretrain_optimizer, run_eagerly=True)
#model.compile(loss=[Clustering_Loss, total_loss], loss_weights=[0.1, 1], optimizer=pretrain_optimizer)

In [ ]:
metrics_list = []
max_ami = 0
for ite in range(int(maxiter)):
    if ite==1:
      # model.load_weights(save_dir + '/Berahmand_text_20NG_n_cluster_12.h5')
      # model.load_weights(save_dir + '/Berahmand_text_REUTERSIDF10K_n_cluster_6.h5')
      # model.load_weights(save_dir + '/Berahmand_text_WEBKB_n_cluster_4.h5')
      model.load_weights(save_dir + '/Berahmand_text_WEBKB_n_cluster_4_500.h5')
      


    if ite % update_interval == 0:

        q, _  = model.predict(x, verbose=0)
        p = target_distribution(q[:,:n_clusters])  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q[:,:n_clusters].argmax(1)
        if y is not None:
            acc = np.round(acc_fun(y, y_pred), 5)
            nmi = np.round(nmi_fun(y, y_pred), 5)
            ari = np.round(ari_fun(y, y_pred), 5)
            ami = np.round(ami_fun(y, y_pred), 5)
            silhouette = np.round(silhouette_score_fun(x, y_pred, random_state=10), 5)
             
            loss = np.round(loss, 5)
            print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f, ami = %.5f, sc = %.5f' % (ite, acc, nmi, ari, ami, silhouette), ' ; loss=', loss)
            metrics_list.append([ite, acc, nmi, ari, ami, silhouette])

        # check stop criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol and acc>=1:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
        
        if ami >= max_ami:
          # model.save_weights(save_dir + '/Berahmand_text_20NG_n_cluster_12_best.h5')
          # model.save_weights(save_dir + '/Berahmand_text_REUTERSIDF10K_n_cluster_6_best.h5')
          # model.save_weights(save_dir + '/Berahmand_text_WEBKB_n_cluster_4_best.h5')
          model.save_weights(save_dir + '/Berahmand_text_WEBKB_n_cluster_4_best_500.h5')
          max_ami = ami

    global idx
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    #print("iter: ",ite,"len(idx) not loss: ",len(idx),", idx: ",idx)
    loss = model.train_on_batch(x=x[idx], y=[np.concatenate((p[idx], np.ones((len(idx),dims[-1]))), axis=1), np.concatenate((x[idx], np.ones((len(idx),dims[-1]))), axis=1)])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0
    # model.save_weights(save_dir + '/Berahmand_text_20NG_n_cluster_12.h5')
    # model.save_weights(save_dir + '/Berahmand_text_REUTERSIDF10K_n_cluster_6.h5')
    # model.save_weights(save_dir + '/Berahmand_text_WEBKB_n_cluster_4.h5')
    model.save_weights(save_dir + '/Berahmand_text_WEBKB_n_cluster_4_500.h5')

### Load the clustering model trained weights

In [ ]:
# model.load_weights(save_dir + '/Berahmand_text_20NG_n_cluster_12_best.h5')
# model.load_weights(save_dir + '/Berahmand_text_REUTERSIDF10K_n_cluster_6_best.h5')
# model.load_weights(save_dir + '/Berahmand_text_WEBKB_n_cluster_4_best.h5')
model.load_weights(save_dir + '/Berahmand_text_WEBKB_n_cluster_4_best_500.h5')

### Final Evaluation

In [ ]:
# Eval.
q,_ = model.predict(x, verbose=0)
p = target_distribution(q[:,:n_clusters])  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q[:,:n_clusters].argmax(1)
if y is not None:
    acc = np.round(acc_fun(y, y_pred), 5)
    nmi = np.round(nmi_fun(y, y_pred), 5)
    ari = np.round(ari_fun(y, y_pred), 5)
    ami = np.round(ami_fun(y, y_pred), 5)
    silhouette = np.round(silhouette_score_fun(x, y_pred, random_state=10), 5)
      
    loss = np.round(loss, 5)
    print('acc = %.5f, nmi = %.5f, ari = %.5f, ami = %.5f, , silhouette_score = %.5f' % (acc, nmi, ari, ami, silhouette), ' ; loss=', loss)

In [ ]:
metrics_list = np.array(metrics_list)
plt.plot(metrics_list[:, 0], metrics_list[:, 2], label='nmi')
plt.show()

In [ ]:
sns.set(font_scale=3)

plt.figure(figsize=(16, 14))
sns.heatmap(confusion_matrix(y, y_pred), annot=True, fmt="d", annot_kws={"size": 20});
plt.title("Confusion matrix", fontsize=30)
plt.ylabel('True label', fontsize=25)
plt.xlabel('Clustering label', fontsize=25)
plt.show()

In [ ]:
extract = Model(inputs = model.input, outputs = model.get_layer(layer_names[len(dims)-1]).output)
z = extract.predict([x])

In [ ]:
sns.set(context="paper", style="white")

reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(z)

fig, ax = plt.subplots(figsize=(12, 10))
color = y.astype(int)
plt.scatter(embedding[:, 0], embedding[:, 1], c=color, cmap="Spectral", s=10)
plt.setp(ax, xticks=[], yticks=[])
plt.show()